## I. Check rarity of getting 4/21'720 Bell's Palsy in 2 month assuming it's from normal population

Modeling a distribution of ratio of people who have got an incidence of Bell's Palsy per year in a sample of size $n$, given that the population has incidence of  $23/100000\,annually = 0.00023\,annually = 0.023\%\,annually$

In [1]:
import random
import math
from scipy import stats

# annual incidence of Bell's palsy
INCIDENCE = 0.00023

For hypothesis testing, we assume that null hypothesis is the following equality:

$ H_0: k=np $  , where:
 - $k$ - number of occurences (successuful tests)
 - $n$ - number of trials
 - $p$ - probability of occurence (successful tests)
 - $np$ - mean (expected value) for the binomial distibution
 
 
 Calculating $p$-value (binomial test):
 
 $ Pr(X=k)=\binom{n}{k}p^k(1-p)^{n-k} $
 
 Calculating $z$-score (for large samples, z-test):
 
 $ Z=\frac{k-np}{\sqrt{np(1-p)}} $

Even though the samples sizes are large, the $H_0$ population probability is extremely small, insomuch that the expected value for $H_0$ populations are close to 0 and often less than 1. Therefore, we should not rely on the normal distribution to approximate such binomial distribution.

In [2]:
### binomial test for vaccine group ###
print("=== vaccine group ===")
k1=4
n1=21720
p1=INCIDENCE/6 # assuming their study had average 2 months of screening

print(f"                        k1 = {k1}")
print(f"                        n1 = {n1}")
print(f"                        p1 = {round(p1*100, 5)}%")
print("")

expected1 = n1*p1
print(f"    expected value = n1*p1 = {round(expected1, 4)}");

# scipy
pval1 = stats.binom_test(x=k1, n=n1, p=p1)
print(f"(scipy)              pval1 = {round(pval1*100, 4)}%")

# naive (binomial test)
pval1 = math.comb(n1, k1) * math.pow(p1,k1) * math.pow(1-p1,n1-k1)
print(f"(naive, binom. test) pval1 = {round(pval1*100, 4)}%")

proportion1 = k1/n1
s_error1 = math.sqrt((proportion1*(1-proportion1))/n1) # standard error for proportion
print(f"                  s_error1 = {round(s_error1,7)}")

print("")


### binomial test for placebo group ###
print("=== placebo group ===")
k2=0
n2=21728
p2=INCIDENCE/6 # assuming their study had average 2 months of screening

print(f"                        k2 = {k2}")
print(f"                        n2 = {n2}")
print(f"                        p2 = {round(p2*100, 5)}%")
print("")

expected2 = n1*p1
print(f"    expected value = n2*p2 = {round(expected2, 4)}");

# scipy
pval2 = stats.binom_test(x=k2, n=n2, p=p2)
print(f"(scipy)              pval2 = {round(pval2*100, 4)}%")

# naive (binomial test)
pval2 = math.comb(n2, k2) * math.pow(p2,k2) * math.pow(1-p2,n2-k2)
print(f"(naive, binom. test) pval2 = {round(pval2*100, 4)}%")

proportion2 = k2/n2
s_error2 = math.sqrt((proportion2*(1-proportion2))/n2) # standard error for proportion
print(f"                  s_error2 = {round(s_error2,7)}")

print("")


=== vaccine group ===
                        k1 = 4
                        n1 = 21720
                        p1 = 0.00383%

    expected value = n1*p1 = 0.8326
(scipy)              pval1 = 1.0385%
(naive, binom. test) pval1 = 0.8707%
                  s_error1 = 9.21e-05

=== placebo group ===
                        k2 = 0
                        n2 = 21728
                        p2 = 0.00383%

    expected value = n2*p2 = 0.8326
(scipy)              pval2 = 100.0%
(naive, binom. test) pval2 = 43.4777%
                  s_error2 = 0.0



There are many methods for computing confidence intervals for proportions.

The most used one is Wald Interval. E.g. for mean of sample the formula is the following:

$95\%CI_{\bar{X}} = \bar{X} \pm z_{95}\sigma_{\bar{X}}$

$99\%CI_{\bar{X}} = \bar{X} \pm z_{99}\sigma_{\bar{X}}$


where standard error for sample mean is calculated as:

$\sigma_{\bar{X}} = \frac{\sigma}{\sqrt{n}}$, where $\sigma$ is standard deviation and $n$ is sample size

<br>
Generally it's of this format:

$95\%CI_{y} = y \pm z_{95}\sigma_y$

for some statistic $y$ and for it's standard error $\sigma_y$ (i.e. standard deviation for $y$ as a measure in the population of measures $y$), where $z_{95}$ is a z-score from normal distribution for two-tailed test with $\alpha$-level $0.05$ ($\alpha$-level $0.01$ for $z_{99}$, and so on).

In python, $z$-score can be calculated using _percent point function_ on the _normal distribution_ provided by a very famous library - _**scipy**_:

```python
from scipy.stats import norm

def zScore_normal(conflevel: float = 0.95):
    z: float = norm.ppf((1+conflevel)/2)
    return abs(z)
```

<br><br>

In this case, for propotion the 99% CI is:

$99\%CI_{p_1} = p_1 \pm z_{99}\sigma_{p_1}$

where $p_1$ is a propotion for the first group, and $\sigma_{p_1}$ is standard error for propotion, $k$ is a number of successfull trials, and $n$ is a total number of trials:

$p_1 = \frac{k}{n}$

$\sigma_{p_1} = \sqrt{\frac{p_1(1-p_1)}{n}}$


<br><br>

General form would look like:

<p style="text-align: center; font-weight: bold">Wald Interval</p>
$$(w^-, w^+) = p\,\pm\,z\sqrt{\frac{p(1-p)}{n}}$$
Which computation can be implemented in Python:

```python
def wald_interval(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using Wald Interval method
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    p = float(x)/n
    sd = math.sqrt((p*(1-p))/n)
    z = zScore_normal(conflevel)
    z_sd = z*sd
    ci = (
        p - z_sd,
        p + z_sd
    )
    return ci
```
<br>

Wald Interval is known for its poor performance with low sample sizes ($n < 100$) or extreme proportions ($p$ close to $0$ or $1$).

Since we have extremely low proportion values with big sample sizes, we can explore what is the best method to use to calculate CI.

For this, let us pre-define a set of different true population proportions. For each of the true population proportions, let's simulate random sampling of size of $n$ trials $50000$ times, and see what part from the constructed CIs of all samples are actually covering (including) the true proportion. Ideally, for a 95% confidence interval, this coverage should be around 95%.


> Source code for this simulation with all mentioned methods in Python:
> 
> https://github.com/Kukuster/CI-for-proportions_methods_analysis
> 
> The original code in R with explanations:
> 
> https://towardsdatascience.com/five-confidence-intervals-for-proportions-that-you-should-know-about-7ff5484c024f

Here is results of a simulation of actual coverage for 95%CI for the sample size of $100$ trials and for proportion values from $0.01$ to $0.99$ using Wald Interval method:

![image](./wald_pfrom0.001_pto0.991_pstep0.01_trials100_samples50000_dark.png "Wald Interval")

What we can see here, is that this method generally underperforms for proportions in range from $0.1$ to $0.9$, and performs terribly bad outside the range.

In this way, simulating enough samples (like $50000$), we can practically compare how precise and how concervative are different methods for calculating confidence intervals for proportions.

Let's take a look at the Wilson Score Interval:

<p style="text-align: center; font-weight: bold">Wilson Score Interval</p>
$$(w^-, w^+) = \frac{p + z^2/2n \pm z\sqrt{p(1-p)/n + z^2/4n^2}}{1+z^2/n}$$

```python
def wilson_score_interval(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using Wilson Score Interval method
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    p = float(x)/n
    z = zScore_normal(conflevel)
    denom = 1 + ((z**2) / n)
    mean = p + ((z**2)/(2*n))
    diff = z * math.sqrt(p*(1-p)/n + (z**2)/(4*n**2))
    ci = (
        (mean-diff)/denom,
        (mean+diff)/denom
    )
    return ci
```

Being a modern improvement on the Wald Interval, it's often used for small samples and extreme probabilities. It doesn't suffer nearly that much from producing a very narrow interval or expanding to close to 100%.

![image](wsi_pfrom0.001_pto0.991_pstep0.01_trials100_samples50000_dark.png "Wilson Score Interval")

We can't say with this, that it underperforms.

People have gone even further, correcting for continuity.

<p style="text-align: center; font-weight: bold">Wilson Score Interval (continuity-corrected)</p>

$$w_{cc}^- = \frac{2np + z^2 - (z\sqrt{z^2 - 1/n + 4np(1-p) + (4p-2)} + 1)}{2(n+z^2)}$$


$$w_{cc}^+ = \frac{2np + z^2 + (z\sqrt{z^2 - 1/n + 4np(1-p) - (4p-2)} + 1)}{2(n+z^2)}$$

or, simplified:

$$e = 2np + z^2;\,\,\, f = z^2 - 1/n + 4np(1-p);\,\,\, g = (4p - 2);\,\,\, h = 2(n+z^2)$$

$$w_{cc}^- = \frac{e - (z\sqrt{f+g} + 1)}{h}$$ 

$$w_{cc}^+ = \frac{e + (z\sqrt{f-g} + 1)}{h}$$

```python
def wilson_score_interval_continuity_corrected(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using Wilson Score Interval method with correction for continuity
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    p = float(x)/n
    z = zScore_normal(conflevel)
    e = 2*n*p + z**2
    f = z**2 - 1/n + 4*n*p*(1-p)
    g = (4*p - 2)
    h = 2*(n+z**2)
    ci = (
        (e - (z*math.sqrt(f+g) + 1))/h,
        (e + (z*math.sqrt(f-g) + 1))/h
    )
    return ci
```

![image](wsicc_pfrom0.001_pto0.991_pstep0.01_trials100_samples50000_dark.png "Wilson Score Interval (continuity-corrected)")

Although the corrected method does deviate more from the 95% point, what it doesn't do, is underestimate the range. Because of that, many would prefer this method.

Using this simulation, we can play around creating our own different methods. E.g. we might be interested in something like Wilson Score Interval with continuity correction, but not that concervative. Let's make up out own method.


<p style="text-align: center; font-weight: bold">Wilson Score Interval (continuity-semi-corrected)</p>
$$(w_{sc}^-, w_{sc}^+) = (\frac{w^- + w_{cc}^-}{2}, \frac{w^+ + w_{cc}^+}{2})$$

```python
def wilson_score_interval_continuity_semicorrected(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using two Wilson Score Interval methods
    (arithmetic mean of ordinary and continuity-corrected methods)
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    uncorrected = wilson_score_interval(x, n, conflevel)
    corrected   = wilson_score_interval_continuity_corrected(x, n, conflevel)
    ci = (
        (corrected[0]+uncorrected[0])/2,
        (corrected[1]+uncorrected[1])/2
    )
    return ci
```
![image](wsisc_pfrom0.001_pto0.991_pstep0.01_trials100_samples50000_dark.png "Wilson Score Interval (continuity-semicorrected)")
This looks better then the continuity-corrected Wilson Score Interval method for proportions in the range from 0.1 to 0.9. There's a good ballance between concervativism and accuracy. Basically, for most values of $p$ coverage stays in range from $94.5\%$ to $97.5\%$.

<br><br>

Using this code we can set up simulation more specifically for our hypothesised and observed proportions between $1\cdot10^{-5}$ to $2\cdot10^{-4}$, and samples sizes between $20000$ and $40000$.

<div style="text-align: center; margin: auto">
    <img style="display: inline-block; margin: 5px" src="./wald_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using Wald Interval for values of p from 1e-5 to 2e-4 and sample size 20000" />
    <img style="display: inline-block; margin: 5px" src="./wald_pfrom1e-06_pto0.000199_pstep0.000001_trials40000_samples50000_dark.png" alt="coverage of 95%CI using Wald Interval for values of p from 1e-5 to 2e-4 and sample size 40000" />
</div>
<div style="text-align: center; margin: auto">
    <img style="display: inline-block; margin: 5px" src="./wsi_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 20000" />
    <img style="display: inline-block; margin: 5px" src="./wsi_pfrom1e-06_pto0.000199_pstep0.000001_trials40000_samples50000_dark.png" alt="coverage of 95%CI using Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 40000" />
</div>
<div style="text-align: center; margin: auto">
    <img style="display: inline-block; margin: 5px" src="./wsicc_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using continuity-corrected Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 20000" />
    <img style="display: inline-block; margin: 5px" src="./wsicc_pfrom1e-06_pto0.000199_pstep0.000001_trials40000_samples50000_dark.png" alt="coverage of 95%CI using continuity-corrected Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 40000" />
</div>
<div style="text-align: center; margin: auto">
    <img style="display: inline-block; margin: 5px" src="./wsisc_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using continuity-semi-corrected Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 20000" />
    <img style="display: inline-block; margin: 5px" src="./wsisc_pfrom1e-06_pto0.000199_pstep0.000001_trials40000_samples50000_dark.png" alt="coverage of 95%CI using continuity-semi-corrected Wilson Score Interval for values of p from 1e-5 to 2e-4 and sample size 40000" />
</div>
<br><br>

Let's choose Wilson Score Interval and Wilson Score Interval continuity-semi-corrected


<br><br><br><br>

<hr>

<!-- <br><br><br><br><br><br>
In our situation, we are interested in samples sizes in around range 20000-40000, and hypothesised probabilities in range from $1\cdot10^{-5}$ to $2\cdot10^{-4}$

<div style="text-align: center; margin: auto">
    <img style="display: inline-block; margin: 5px" src="./wald_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using Wald Interval for values of p from 1e-5 to 2e-4" />
    <img style="display: inline-block; margin: 5px" src="./wald_pfrom1e-06_pto0.000199_pstep0.000001_trials20000_samples50000_dark.png" alt="coverage of 95%CI using Wald Interval for values of p from 1e-5 to 2e-4" />
</div>

<br><br> -->

Let's calculate _standard error for proportion_ for both groups


In [3]:
from scipy.stats import norm
from typing import Tuple, Union, List

def zScore_normal(conflevel: float = 0.95):
    z: float = norm.ppf((1+conflevel)/2)
    return abs(z)


def wilson_score_interval(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using Wilson Score Interval method
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    p = float(x)/n
    z = zScore_normal(conflevel)
    denom = 1 + ((z**2) / n)
    mean = p + ((z**2)/(2*n))
    diff = z * math.sqrt(p*(1-p)/n + (z**2)/(4*n**2))
    ci = (
        (mean-diff)/denom,
        (mean+diff)/denom
    )
    return ci

def wilson_score_interval_continuity_corrected(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using Wilson Score Interval method with correction for continuity
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    p = float(x)/n
    z = zScore_normal(conflevel)
    e = 2*n*p + z**2
    f = z**2 - 1/n + 4*n*p*(1-p)
    g = (4*p - 2)
    h = 2*(n+z**2)
    ci = (
        (e - (z*math.sqrt(f+g) + 1))/h,
        (e + (z*math.sqrt(f-g) + 1))/h
    )
    return ci

def wilson_score_interval_continuity_semicorrected(x: int, n: int, conflevel: float = 0.95):
    """Calculates confidence interval for proportions using two Wilson Score Interval methods
    (arithmetic mean of ordinary and continuity-corrected methods)
    
    `x` - succeeded trials
    
    `n` - total trials
    
    `conflevel` - confidence level (0 < float < 1)
    """
    uncorrected = wilson_score_interval(x, n, conflevel)
    corrected   = wilson_score_interval_continuity_corrected(x, n, conflevel)
    ci = (
        (corrected[0]+uncorrected[0])/2,
        (corrected[1]+uncorrected[1])/2
    )
    return ci


def pretty_CI_percent(ci: Tuple[float, float]) -> Tuple[str, str]:
    frmt = lambda flt: '{0:.10f} %'.format(flt*100)
    pretty_CI = (frmt(ci[0]), frmt(ci[1]))
    return pretty_CI

def std_err(p: float, n: int) -> float:
    """Calculates standard error for proportion
    
    p - proportion
    
    n - sample size
    """
    return math.sqrt((p*(1-p)/n))

print("=== Standard error for proportions for vaccine group ===")
SE_proportion1 = std_err(proportion1, n1)
print(f"for:")
print(f"proportion1 = {round(proportion1, 5)}, n1 = {round(n1, 5)}")
print(f"SE_proportion1 = {round(SE_proportion1,6)}")
print("")

print("=== Standard error for proportions for placebo group ===")
SE_proportion2 = std_err(proportion2, n2)
print(f"for:")
print(f"proportion2 = {round(proportion2, 5)}, n2 = {round(n2, 5)}")
print(f"SE_proportion2 = {round(SE_proportion2,6)}")


=== Standard error for proportions for vaccine group ===
for:
proportion1 = 0.00018, n1 = 21720
SE_proportion1 = 9.2e-05

=== Standard error for proportions for placebo group ===
for:
proportion2 = 0.0, n2 = 21728
SE_proportion2 = 0.0


What's more important, is standard error for the difference between two population proportions:

$\sigma_{diff} = \sqrt{\sigma_{p_1}^2 + \sigma_{p_2}^2}$

In [4]:
print("=== Standard error for difference of proportions ===")

SE_proportions_diff = math.sqrt(SE_proportion1**2 + SE_proportion2**2)
print(f"SE_proportions_diff = {round(SE_proportions_diff, 6)}")

=== Standard error for difference of proportions ===
SE_proportions_diff = 9.2e-05


<!-- Now we can calculate 95%CI and 99%CI for each sample, to figure out were their respective population proportion lies within for certain -->

<< _The point of considering a confidence interval is that if the metric is the __relative risk__ and the 95
percent CI includes 1.0, then any difference between treatments is probably due to chance (i.e.,
the p value would be > 0.05), Similarly, if the metric is the __absolute difference__ and the 95 percent
include 0.0, there is no real difference between treatments. However, the upper and lower limits
of the confidence interval will give an indication of what the maximum and minimum
differences might be and therefore a clinical judgment can be made about the importance of the
difference if it exists._ >>

Absolute risk difference (differenece between sample proportions):

$r_{abs\Delta} = \frac{k_1}{n_1} - \frac{k_2}{n_2}$

In [5]:
absolute_risk_difference = (k1/n1) - (k2/n2)
print(f"absolute_risk_difference = {round(absolute_risk_difference, 6)}")

absolute_risk_difference = 0.000184


Finally, lets calculate 95%CI and 99%CI for each proportion and for the absolute proportion difference 

In [7]:
print("=== CI for proportion (vaccine group) ===")
proportion1_95CI = wilson_score_interval(k1, n1, 0.95)
proportion1_95CI_pretty = pretty_CI_percent(proportion1_95CI)
print(f"proportion1_95CI = {proportion1_95CI_pretty}")
proportion1_99CI = wilson_score_interval(k1, n1, 0.99)
proportion1_99CI_pretty = pretty_CI_percent(proportion1_99CI)
print(f"proportion1_99CI = {proportion1_99CI_pretty}")
print("")

print("=== CI for proportion (placebo group) ===")
proportion2_95CI = wilson_score_interval(k2, n2, 0.95)
proportion2_95CI_pretty = pretty_CI_percent(proportion2_95CI)
print(f"proportion2_95CI = {proportion2_95CI_pretty}")
proportion2_99CI = wilson_score_interval(k2, n2, 0.99)
proportion2_99CI_pretty = pretty_CI_percent(proportion2_99CI)
print(f"proportion2_99CI = {proportion2_99CI_pretty}")
print("")

print("=== CI for difference of proportion ===")
proportions_diff_95CI = (absolute_risk_difference - z95*SE_proportions_diff, absolute_risk_difference + z95*SE_proportions_diff)
proportions_diff_95CI_pretty = ('{0:.10f}'.format(proportions_diff_95CI[0]*100)+" %", '{0:.10f}'.format(proportions_diff_95CI[1]*100)+" %")
print(f"proportions_diff_95CI = {proportions_diff_95CI_pretty}")
proportions_diff_99CI = (absolute_risk_difference - z99*SE_proportions_diff, absolute_risk_difference + z99*SE_proportions_diff)
proportions_diff_99CI_pretty = ('{0:.10f}'.format(proportions_diff_99CI[0]*100)+" %", '{0:.10f}'.format(proportions_diff_99CI[1]*100)+" %")
print(f"proportions_diff_99CI = {proportions_diff_99CI_pretty}")


=== CI for proportion (vaccine group) ===
proportion1_95CI = ('0.0071619287 %', '0.0473471177 %')
proportion1_99CI = ('0.0054791992 %', '0.0618800429 %')

=== CI for proportion (placebo group) ===
proportion2_95CI = ('0.0000000000 %', '0.0176766374 %')
proportion2_99CI = ('0.0000000000 %', '0.0305268372 %')

=== CI for difference of proportion ===


NameError: name 'z95' is not defined

Interestingly enought, 95%CI indicating statistically significant effect may represent public concern regarding the data about BP.

Indeed, 99%CI has to be much more appropriate to assume here, because there are plenty of other diseases that could spike to such difference.

99%CI for difference of proportions including zero indicates statistically insignificant difference between two proportions.

 ## II. Check rarity of getting a sample of 37'706 people with 0.3% (118) prevalence of rheumatic diseases from a normal population (about 1.5% prevalence)

In [ ]:
PREVALENCE = 0.015

### binomial test ###
k=118
n=37706

print(f"PREVALENCE = {PREVALENCE}");
print(f"         k = {k}")
print(f"         n = {n}")
print("")
expected = n*PREVALENCE
print(f"expected value = n*PREVALENCE = {round(expected, 4)}");

proportion=k/n

pval = stats.binom_test(x=k, n=n, p=PREVALENCE)
print(f"    pval = {pval}")

s_error = math.sqrt((proportion*(1-proportion))/n) # standard error for proportion
print(f" s_error = {round(s_error,7)}")


For prevalence $1.5\%$ the resulting $p$-value ranges somewhere between $10^{-117}$ to $10^{-116}$ depending on the calculation

For prevalence $1.2\%$: $p$-value $=$ from $10^{-79}$ to $10^{-78}$

For prevalence $1\%$: $p$-value $=$ from $10^{-55}$ to $10^{-54}$

So that's unmeasurably small numbers.


What are the possible adequate prevalence for the biased population from which people were picked?

For prevalence $0.8\%$: $p$-value $=$ from $10^{-34}$ to $10^{-33}$

For prevalence $0.6\%$: $p$-value $=$ from $10^{-16}$ to $3\cdot10^{-15}$

For prevalence $0.5\%$: $p$-value $=$ from $10^{-8}$ to $4\cdot10^{-8}$

For prevalence $0.4\%$: $p$-value $= 0.006239 = 0.6239\%$


Another way to assess possible prevalence for the biased population would be to calculate CI for proportion:

In [ ]:
print("=== CI for proportion of people with rheumatic disease ===")
rheu_proportion_95CI = (proportion - z95*s_error, proportion + z95*s_error)
rheu_proportion_95CI_pretty = ('{0:.10f}'.format(rheu_proportion_95CI[0]*100)+" %", '{0:.10f}'.format(rheu_proportion_95CI[1]*100)+" %")
print(f"rheu_proportion_95CI = {rheu_proportion_95CI_pretty}")
rheu_proportion_99CI = (proportion - z99*s_error, proportion + z99*s_error)
rheu_proportion_99CI_pretty = ('{0:.10f}'.format(rheu_proportion_99CI[0]*100)+" %", '{0:.10f}'.format(rheu_proportion_99CI[1]*100)+" %")
print(f"rheu_proportion_99CI = {rheu_proportion_99CI_pretty}")
print("")


## III. If step II rarity is significant then assume their sample is indeed from a biased distribution and has 0.3% prevalence of rheumatic diseases

We can safely now use the number $0.3\%$ for the biases population prevalence of rheumatic disease. Or we can toughly round the number towards less effect to $0.4\%$.


## IV. Extrapolate the bias towards people with less rheumatic disease to the bias towards less allergic/autoimmune disease of their sample to a new expected incidence of Bell's palsy

Naively, this can be done by taking the same bias proportion to the BP incidence as it is for rheumatic disease prevalence.

That is, if the world population has about $1.2\%$ prevalence of rheumatic disease, then:

$0.023\% \cdot \frac{0.4\%}{1.2\%} = 0.023\% : 3 = 0.0077\% = 0.000077$

Assuming incidence of Bell's palsy in a biased population to be $0.0077\%$


## V. Check rarity of getting 4/21'720 Bell's Palsy in 2 month assuming it's from the biased distribution from step III (assuming extrapolation from IV)

In [ ]:
INCIDENCE_biased = 0.000077

### binomial test for vaccine group ###
print("=== vaccine group ===")
k1=4
n1=21720
p1=INCIDENCE_biased/6 # assuming their study had average 2 months of screening

print(f"                        k1 = {k1}")
print(f"                        n1 = {n1}")
print(f"                        p1 = {round(p1*100, 5)}%")
print("")

expected1 = n1*p1
print(f"    expected value = n1*p1 = {round(expected1, 4)}")

# scipy
pval1 = stats.binom_test(x=k1, n=n1, p=p1)
print(f"(scipy)              pval1 = {round(pval1*100, 4)}%")

# naive (binomial test)
pval1 = math.comb(n1, k1) * math.pow(p1,k1) * math.pow(1-p1,n1-k1)
print(f"(naive, binom. test) pval1 = {round(pval1*100, 4)}%")

proportion1 = k1/n1
s_error1 = math.sqrt((proportion1*(1-proportion1))/n1) # standard error for proportion
print(f"                  s_error1 = {round(s_error1,7)}")

print("")


### binomial test for placebo group ###
print("=== placebo group ===")
k2=0
n2=21728
p2=INCIDENCE_biased/6 # assuming their study had average 2 months of screening

print(f"                        k2 = {k2}")
print(f"                        n2 = {n2}")
print(f"                        p2 = {round(p2*100, 5)}%")
print("")

expected2 = n2*p2
print(f"    expected value = n2*p2 = {round(expected2, 4)}")

# scipy
pval2 = stats.binom_test(x=k2, n=n2, p=p2)
print(f"(scipy)              pval2 = {round(pval2*100, 4)}%")

# naive (binomial test)
pval2 = math.comb(n2, k2) * math.pow(p2,k2) * math.pow(1-p2,n2-k2)
print(f"(naive, binom. test) pval2 = {round(pval2*100, 4)}%")

proportion2 = k2/n2
s_error2 = math.sqrt((proportion2*(1-proportion2))/n2) # standard error for proportion
print(f"                  s_error2 = {round(s_error2,7)}")

print("")


In [ ]:
SE_proportion1 = s_error1
SE_proportion2 = s_error2

In [ ]:
print("=== Standard error for difference of proportions ===")

SE_proportions_diff = math.sqrt(math.pow(SE_proportion1,2) + math.pow(SE_proportion2,2))
print(f"SE_proportions_diff = {round(SE_proportions_diff, 6)}")

In [ ]:
absolute_risk_difference = (k1/n1) - (k2/n2)
print(f"absolute_risk_difference = {round(absolute_risk_difference, 6)}")

In [ ]:
print("=== CI for proportion (vaccine group) ===")
proportion1_95CI = (proportion1 - z95*SE_proportion1, proportion1 + z95*SE_proportion1)
proportion1_95CI_pretty = ('{0:.10f}'.format(proportion1_95CI[0]*100)+" %", '{0:.10f}'.format(proportion1_95CI[1]*100)+" %")
print(f"proportion1_95CI = {proportion1_95CI_pretty}")
proportion1_99CI = (proportion1 - z99*SE_proportion1, proportion1 + z99*SE_proportion1)
proportion1_99CI_pretty = ('{0:.10f}'.format(proportion1_99CI[0]*100)+" %", '{0:.10f}'.format(proportion1_99CI[1]*100)+" %")
print(f"proportion1_99CI = {proportion1_99CI_pretty}")
print("")

print("=== CI for proportion (placebo group) ===")
proportion2_95CI = (proportion2 - z95*SE_proportion2, proportion2 + z95*SE_proportion2)
proportion2_95CI_pretty = ('{0:.10f}'.format(proportion2_95CI[0]*100)+" %", '{0:.10f}'.format(proportion2_95CI[1]*100)+" %")
print(f"proportion2_95CI = {proportion2_95CI_pretty}")
proportion2_99CI = (proportion2 - z99*SE_proportion2, proportion2 + z99*SE_proportion2)
proportion2_99CI_pretty = ('{0:.10f}'.format(proportion2_99CI[0]*100)+" %", '{0:.10f}'.format(proportion2_99CI[1]*100)+" %")
print(f"proportion2_99CI = {proportion2_99CI_pretty}")
print("")

print("=== CI for difference of proportions ===")
proportions_diff_95CI = (absolute_risk_difference - z95*SE_proportions_diff, absolute_risk_difference + z95*SE_proportions_diff)
proportions_diff_95CI_pretty = ('{0:.10f}'.format(proportions_diff_95CI[0]*100)+" %", '{0:.10f}'.format(proportions_diff_95CI[1]*100)+" %")
print(f"proportions_diff_95CI = {proportions_diff_95CI_pretty}")
proportions_diff_99CI = (absolute_risk_difference - z99*SE_proportions_diff, absolute_risk_difference + z99*SE_proportions_diff)
proportions_diff_99CI_pretty = ('{0:.10f}'.format(proportions_diff_99CI[0]*100)+" %", '{0:.10f}'.format(proportions_diff_99CI[1]*100)+" %")
print(f"proportions_diff_99CI = {proportions_diff_99CI_pretty}")

The p-value for getting total of 4 cases from total of 43'448 people is $0.259\%$

Then, from that, there's $1$ in $2^4$ probability of getting all these cases in a one specific group only by chance, that is $\frac{1}{2^4} = \frac{1}{16} = 0.0625 = 6.25\%$. Therefore, the fact that 4 cases were in one group and 0 in another is a statistically insignificant indicator of significant inclination towards vaccine group. This is also idicated with 95%CI of the difference between two proportions almost touching 0, and 99%CI including it.

On the other hand, the plain fact itself that 4 cases took place during the study is a significant indicator of the presence of the effect:

$pvalue_{total} = 0.2589\%$

In [ ]:
kt=k1+k2
nt=n1+n2
pt=INCIDENCE_biased/6 # assuming their study had average 2 months of screening

print(f"                        kt = {kt}")
print(f"                        nt = {nt}")
print(f"                        pt = {round(pt*100, 5)}%")
print("")

expectedt = nt*pt
print(f"    expected value = nt*pt = {round(expectedt, 4)}")

# scipy
pvalt = stats.binom_test(x=kt, n=nt, p=pt)
print(f"(scipy)              pvalt = {round(pvalt*100, 4)}%")

# naive (binomial test)
pvalt = math.comb(nt, kt) * math.pow(pt,kt) * math.pow(1-pt,nt-kt)
print(f"(naive, binom. test) pvalt = {round(pvalt*100, 4)}%")

proportion_t = kt/nt
s_errort = math.sqrt((proportion_t*(1-proportion_t))/nt) # standard error for proportion
print(f"                  s_errort = {round(s_errort,7)}")

print("")


## VI. Calculate different effect size formulas to show practical significance of the effect. Some: relative risk, absolute risk difference, number needed to vaccinate, odds ratio

To calculate various effect size formulas, we'll use expected value for the placebo group event rate

$biased\,incidence \times n_{placebo} = 0.00128\% \times 21728 = 0.2788$

intead of actual observed $0$ cases


Thus, instead of this data:

| Group        | had Bell's palsy ($k_i$) | didn't have ($n_i - k_i$) | total ($n_i$) |
| ------------ |:------------------------:|:-------------------------:|:-------------:|
| vaccine      |             4            |           21716           |     21720     |
| placebo      |             0            |           21728           |     21728     |


we'll use this corrected one:

| Group        | had Bell's palsy ($k_i$) | didn't have ($n_i - k_i$) | total ($n_i$) |
| ------------ |:------------------------:|:-------------------------:|:-------------:|
| vaccine      |             4            |           21716           |     21720     |
| placebo      |           0.2788         |           21727.7212      |     21728     |




vaccine (experimental) group event rate:

$r_1 = \frac{4}{21720} = 0.000184 = 0.0184\%$

placebo (control) group event rate:

$r_2 = \frac{0.2788}{21728} = 0.0000128 = 0.00128\%$

<br>

relative risk of bell's palsy (the ratio of the experimental event rate to the control event rate):

$\frac{r_1}{r_2} = \frac{0.0184\%}{0.00128\%} = 14.375 = 1437.5\%$

absolute risk difference (the difference between risks, the difference between experimental event rate to the control event rate):

$r_1 - r_2 = 0.0184\% - 0.00128\% = 0.01712\% = 0.0001712$

Therefore, having the vaccine increases changes of getting Bell's palsy by $1437.5\%$ (i.e. $14.375$ times), that is plus $0.01712\%$ in absolute

<br>

number of patients needed to vaccinate to cause 1 incidence of Bell's palsy (1 divided by absolute risk difference):

$\frac{1}{0.0001712} = 5841.12$





## Sources:
 - https://www.who.int/medicines/technical_briefing/tbs/03-PG_Assessing-drug-efficacy_final-08.pdf (specifically, pages 23-27)
 - https://online.stat.psu.edu/stat100/book/export/html/678
 - https://towardsdatascience.com/five-confidence-intervals-for-proportions-that-you-should-know-about-7ff5484c024f
 - Wallis, Sean A. (2013). "Binomial confidence intervals and contingency tests: mathematical fundamentals and the evaluation of alternative methods" [(PDF)](https://www.ucl.ac.uk/english-usage/staff/sean/resources/binomialpoisson.pdf). Journal of Quantitative Linguistics. 20 (3): 178–208. doi:10.1080/09296174.2013.799918.